# Generisches Programmieren mit Java

#### Marcel Lüthi
#### Departement Mathematik und Informatik <br/> Universität Basel

# Agenda

* Problemmotivation
* Lösung: Generische Programmieren mit Objekten (Old Style)
* Intermezzo: Hüllenklassen
* Kollektionen von Objekten verschiedenen Typs
* Operationen auf Kollektionen von Objekten



# Ausgangslage: Stack für Integers

In [ ]:
class IntStack {
    private int[] data = new int[1000]; // Vereinfachung: Feste Grösse
    private int nElements = 0; // Anzahl Elemente im Stack
    
    public void push(int i) {
        data[nElements] = i;     
        nElements += 1;
    }
    
    public int pop() { 
        if (nElements <= 0) {
            throw new RuntimeException("pop from empty stack");
        }
        else {
            nElements -= 1;
            return data[nElements];
       }
    }
    public int size() { return this.nElements; }
}

# Nutzung vom Stack

In [ ]:
IntStack stack = new IntStack();
System.out.println("Size " + stack.size());
stack.push(10);
System.out.println("Size " + stack.size());
int element = stack.pop();
System.out.println("gelesen: " + element);
System.out.println("Size " + stack.size());

# Problem

* Was machen wir wenn wir nun einen Stack von Strings wollen?
* ... und einen Stack von Double?
* ... und einen Stack von Floats?

> Kopieren von Code ist keine Lösung!

# Generische Programmierung mit Objekten 

# Ansatz

* Ausnutzen, der Klassenhierarchie

![hierarchy](./images/class-hierarchy.gif)
<figcaption style="font-size:xx-small">Quelle: https://docs.oracle.com/javase/tutorial/figures/java/classes-object.gif</figcaption>

# Ansatz

* Zuweisung zu Objekt ist immer möglich

In [ ]:
String s = new String("");
Integer i = new Integer(5);
Date d = new Date(1970, 12, 31);

# Generisches Stack Implementation

In [ ]:
class Stack { 
    
    private Object[] data = new Object[1000]; // Vereinfachung: Feste grösse
    private int nElements = 0; // Anzahl Elemente im Stack
 
    public void push(Object element) {
        data[nElements] = element;     
        nElements += 1;
    }
    
    public Object pop() { 
        if (nElements <= 0) {
            return null;
        }
        else {
            nElements -= 1;
            return data[nElements];
       }
    }
    public int size() { return this.nElements; }
}


# Übung: Nutzung des Stacks


* Nutzen Sie den generischen Stack. 
* Speichern Sie ein Objekt einer von Ihnen erstellten Klasse


# Intermezzo: Hüllenklassen und autoboxing

# Stack mit primitiven Datentypen

Beobachtung: Wir können nur Objekte auf dem Stack speichern.

* Ist int ein Objekt?
* Funktioniert folgender Code?

In [ ]:
int anInt  = 3;
Stack stack = new Stack();
stack.push(anInt);

# Hüllenklassen für Primitive Datentypen

* Zu jedem primitiven Datentyp existiert eine Hüllenklasse (Wrapper)

| Primitiver Typ | Hüllenklasse |
|----------------|--------------|
| int | Integer |
| short | Short |
| long | Long | 
| double | Double |
| float | Float | 
| char | Character | 
| boolean | Boolean |

Beispiel:

In [ ]:
Integer anInt = new Integer(5);
int aPrimitiveInt = anInt.intValue();    

# Autoboxing

* *Boxing*: Ein primitiver Typ wird automatisch in ein Objekt der Hüllenklasse konvertiert
* *Unboxing*: Ein Hüllenklassentyp wird automatisch in einen primitiven Typen konvertiert


In [ ]:
Integer aBoxedInt = 5;
int anUnboxedInt = new Integer(5)

# Gotcha: Gleichheit

* (Un)boxing macht Equality noch etwas schwieriger.

In [ ]:
System.out.println(new Integer(5) == new Integer(5));
System.out.println(5 == new Integer(5));
System.out.println(new Integer(5) == 5);
System.out.println(new Integer(5).equals(5));

# Kollektionen von Objekten mit verschiedenen Typen

# Stacks mit verschiedenen Typen

* Alle Klassen sind Subklassen von Objekt!
    * Wir können Objekte von beliebigen Typen speichern

In [ ]:
Stack createMixedStack() { 

    Stack stack = new Stack();
    stack.push("a String");
    stack.push(new Integer(5));
    stack.push(new java.util.Date());
    stack.push(java.awt.Color.GREEN);

    return stack;
}

createMixedStack();


# Übung: Ausdrucken aller Elemente

* Weshalb funktioniert das?

# Arbeiten mit Elementen

* Bisher: Arbeit nur mit Methoden von ```Object```
* Wie arbeiten wir mit den Methoden der einzelnen Klassen


In [ ]:
Object o = new String("hello world");
o.replace("world", "class");

* Lösung: ```instanceof``` Operator

# Arbeiten mit Elementen in Kollektionen

In [ ]:
Stack stack = createMixedStack();

while (stack.size() > 0) {
    Object element = stack.pop();
    
    if (element instanceof String) {
        String s = (String) element;
        System.out.println("got string, which starts with the letter " +s.charAt(0));
    }
    else if (element instanceof Date) {
        Date d = (Date) element;
        System.out.println("The year is "+ d.getYear());
    }
    else { 
        System.out.println("I don't know what to do with this type of object");
    }
}


# Operationen auf Kollektionen von generischen Objekten   

# Hilfsfunktion

* Erzeugen von Arrays aus 3 ```Objects```

In [ ]:
Object[] createArrayFromObjects(Object o1, Object o2, Object o3) { 
    Object[] a = new Object[3];
    a[0] = o1; 
    a[1] = o2;
    a[2] = o3;
    return a;    
}

* Beispiel:

In [ ]:
createArrayFromObjects("String", 5, new Date(1970, 12, 31))

# Übung: Suchen von Objekt

* Schreiben Sie eine Funktion zum Suchen von Objekten die folgende Spezifikation erfüllt.

In [ ]:
// Ihre Funktion

In [ ]:

Object[] array = createArrayFromObjects("String 1", new Date(1970, 12, 31), "String 2" );
findObjectInArray(array, new Date(1970, 12, 31)); // Output 1
findObjectInArray(array, new String("String 3")); // Output -1

# Sortieren von Arrays

* Sortieren heisst, wir müssen Objekte vergleichen können
    * Was ist grösser, "abc" oder "def"?
    * Was ist grösser, ```Color.GREEN``` oder ```Color.RED```    
    

In [ ]:
Object o1 = new Integer(5);
Object o2 = "a String";
o1 < o2;

# Idee: Gemeinsame Operation: CompareTo

In [ ]:
interface CanCompare { 
   // returns -1, 0, 1
    int compareTo(Object o);
}



# Beispielklasse Tupel:

In [ ]:

public class Tuple implements CanCompare { 

    int first;
    int second;

    public Tuple (int first, int second) {
        this.first = first;
        this.second = second;
    }
    
    public int compareTo(Object thatObject) { 
        // not implemented yet
        return 0;
    }
    
    public String toString() { 
        return "(" +this.first + ", " + this.second +")";
    }
    
}

# Generische Sortierfunktion

In [ ]:
void sort(CanCompare[] arr) { 
    for (int i = 0; i < arr.length ; ++i) {
        int j = i;
        while(j > 0) {                
            if (arr[j].compareTo(arr[j-1]) < 0) {            
                CanCompare temp = arr[j];
                arr[j] = arr[j-1];
                arr[j-1] = temp;
            }
            j--;
        }
    }    
}


# Generische Sortierfunktion

In [ ]:
CanCompare[] array = {new Tuple(5,1), new Tuple(1,7), new Tuple(2,8), new Tuple(2,6)};
sort(array);
for (int i = 0; i < array.length; ++i) {
    System.out.println(array[i]);
}

#### Übung

* Ändern Sie den Code so ab, das zuerst nach dem zweiten Tupelelement sortiert wird.

# Probleme von generischem Programmieren mit Objekten

* Wir brauchen explizite downcasts

```String a = (String) stack.pop() // unschön``` 

* Intent nicht klar
    * Speichern wir Objekte oder Listen oder strings?

```
void foo(Object[] objectArray)  {

    // was ist in Array drin

}
```

# Probleme von generischem Programmieren mit Objekten (2)
* Nicht typesafe
    * Fehler zur Laufzeit - Keine Hilfe von Compiler


``` String a =  (String) stack.pop() ```

* Klassen müssen in Hierarchie gebracht werden
    * Nicht möglich ohne Source Code

# Ausblick

* Bessere Lösung mit Java Generics
    * Teil von Java seit Java 5
* Wichtiges Element von fast allen Programmiersprachen